In [44]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
pd.options.display.max_columns=None
from datetime import datetime as dt
import glob
import shutil
%config InteractiveShell.ast_node_interactivity = 'all'
#import dependencies

In [45]:

df_list=[]
try:
  for file in glob.glob('**/yellow_t*.parquet'):
    df=pd.read_parquet(file, engine="pyarrow")
    df_list.append(df)
  df=pd.concat(df_list)
except:
  print("No hay archivos para cargar")


se agrego la columna IdFecha a la funcion t_datime

In [48]:
def t_datime(df):
    
    df = df[df["tpep_pickup_datetime"] >= '2018-01-01 00:00:00']
    df = df[df["tpep_pickup_datetime"] <= dt.now()]
    df['IdSemana']= df['tpep_pickup_datetime'].dt.week
    df["IdFecha"] = df['tpep_pickup_datetime'].dt.strftime('%Y%m%d%H')
    df.IdFecha = df.IdFecha.astype("int")
    df.drop(columns=["congestion_surcharge","airport_fee","store_and_fwd_flag"], inplace=True)
    df.sort_values(by='tpep_pickup_datetime', ascending=True, inplace=True)
    return df

def t_int(df):
    df['passenger_count'] = pd.to_numeric(df['passenger_count'], downcast="integer" )
    df['RatecodeID'] = pd.to_numeric(df['RatecodeID'], downcast="integer" )
    df['PULocationID'] = pd.to_numeric(df['PULocationID'], downcast="integer" )
    df['DOLocationID'] = pd.to_numeric(df['DOLocationID'], downcast="integer" )
    df['payment_type'] = pd.to_numeric(df['payment_type'], downcast="integer" )
    df['RatecodeID'] = pd.to_numeric(df['RatecodeID'], downcast="integer" )
    df['RatecodeID'] = pd.to_numeric(df['RatecodeID'], downcast="integer" )
    return df

def t_float(df):
    df['trip_distance'] = pd.to_numeric(df['trip_distance'], downcast="float" )
    df['fare_amount'] = pd.to_numeric(df['fare_amount'], downcast="float" )
    df['extra'] = pd.to_numeric(df['extra'], downcast="float" )
    df['mta_tax'] = pd.to_numeric(df['mta_tax'], downcast="float" )
    df['tip_amount'] = pd.to_numeric(df['tip_amount'], downcast="float" )
    df['tolls_amount'] = pd.to_numeric(df['tolls_amount'], downcast="float" )
    df['improvement_surcharge'] = pd.to_numeric(df['improvement_surcharge'], downcast="float" )
    df['total_amount'] = pd.to_numeric(round(df['total_amount'] ))
    return df

def total_transform(df):
    df = t_datime(df)
    df = t_int(df)
    df = t_float(df)
    return df

df =  total_transform(df)

C:\Users\andre\AppData\Local\Temp\ipykernel_11468\4200162722.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['IdSemana']= df['tpep_pickup_datetime'].dt.week


In [49]:
df_Impar = df

In [50]:
def boroughs (df_Impar):
    #se carga el csv con las zonas
    zona = pd.read_csv("taxi+_zone_lookup.csv")
    #se borran las columnas que no usamos
    zona.drop(columns=["Zone","service_zone"], inplace=True)
    #cambiamos el nombre para poder hacer el merge
    zona.rename(columns={"LocationID": "PULocationID"},inplace=True)
    #se hace el merge y se crea la columnna resultante del merge "Borough"
    df_Impar = pd.merge(df_Impar,zona, how="left", on=["PULocationID"])
    #se le cambia el nombre a "boroug" para que haga referencia al borough de partida
    df_Impar.rename(columns={"Borough": "PUBorough"},inplace=True)
    #mismo procedimiento para borough de llegada
    zona.rename(columns={"PULocationID": "DOLocationID"},inplace=True)
    df_Impar = pd.merge(df_Impar,zona, how="left", on=["DOLocationID"])
    df_Impar.rename(columns={"Borough": "DOLocation"},inplace=True)
    return df_Impar

df_Impar = boroughs(df_Impar)

Se cambian los borough por los idBorough

In [51]:
df_Impar['PUBorough']=np.where(df_Impar['PUBorough']=="EWR",1,df_Impar['PUBorough'])
df_Impar['PUBorough']=np.where(df_Impar['PUBorough']=="Queens",2,df_Impar['PUBorough'])
df_Impar['PUBorough']=np.where(df_Impar['PUBorough']=="Bronx",3,df_Impar['PUBorough'])
df_Impar['PUBorough']=np.where(df_Impar['PUBorough']=="Manhattan",4,df_Impar['PUBorough'])
df_Impar['PUBorough']=np.where(df_Impar['PUBorough']=="Staten Island",5,df_Impar['PUBorough'])
df_Impar['PUBorough']=np.where(df_Impar['PUBorough']=="Brooklyn",6,df_Impar['PUBorough'])
df_Impar['PUBorough']=np.where(df_Impar['PUBorough']=="Unknown",7,df_Impar['PUBorough'])

df_Impar.rename(columns={"DOLocation":"DOBorough"},inplace=True)
df_Impar['DOBorough']=np.where(df_Impar['DOBorough']=="EWR",1,df_Impar['DOBorough'])
df_Impar['DOBorough']=np.where(df_Impar['DOBorough']=="Queens",2,df_Impar['DOBorough'])
df_Impar['DOBorough']=np.where(df_Impar['DOBorough']=="Bronx",3,df_Impar['DOBorough'])
df_Impar['DOBorough']=np.where(df_Impar['DOBorough']=="Manhattan",4,df_Impar['DOBorough'])
df_Impar['DOBorough']=np.where(df_Impar['DOBorough']=="Staten Island",5,df_Impar['DOBorough'])
df_Impar['DOBorough']=np.where(df_Impar['DOBorough']=="Brooklyn",6,df_Impar['DOBorough'])
df_Impar['DOBorough']=np.where(df_Impar['DOBorough']=="Unknown",7,df_Impar['DOBorough'])


df_Impar = df_Impar[df_Impar["PUBorough"] != 7]
df_Impar = df_Impar[df_Impar["DOBorough"] != 7]

In [52]:
df_Impar.drop(columns={"PULocationID","DOLocationID"},inplace=True)

In [53]:
def neg_values(df_Impar):   
    df_Impar = df_Impar.drop(df_Impar[df_Impar.fare_amount<0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.trip_distance<0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.total_amount<0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.RatecodeID==6].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.RatecodeID==99].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.trip_distance==0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.fare_amount==0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.total_amount==0].index)
    return df_Impar

df_Impar=neg_values(df_Impar)

In [54]:
def t_outliers(column):
    Q1 = df_Impar[column].quantile(0.01)
    Q3 = df_Impar[column].quantile(0.99)
    #Se calcula el rango intercuartilico IQR.
    IQR = Q3 -Q1
    outliers_Sup_trip = (Q3 + (1.5*IQR)) 
    return outliers_Sup_trip

trip_distance = t_outliers("trip_distance")
fare_amount = t_outliers("fare_amount")
tip_amount = t_outliers("tip_amount")
total_amount = t_outliers("total_amount")

df_Impar["outlier"] = df_Impar.apply(lambda row: 1 if row["trip_distance"]>trip_distance or row["fare_amount"]>fare_amount or
row["total_amount"]>total_amount or row["tip_amount"]>tip_amount else 0, axis=1)

normalizacion de columnas.

In [55]:
df_Impar.rename(columns={"VendorID": "IdVendor","RatecodeID":"IdRatecode","payment_type":"IdPayment_Type",
"PUBorough":"IdPUBorough","DOBorough":"IdDOBorough" },inplace=True)

In [56]:
df_Impar['IdPUBorough'] = pd.to_numeric(df_Impar['IdPUBorough'], downcast="integer" )
df_Impar['IdDOBorough'] = pd.to_numeric(df_Impar['IdDOBorough'], downcast="integer" )
df_Impar['outlier'] = pd.to_numeric(df_Impar['outlier'], downcast="integer" )

Se terminan los cambios en df_Impar y queda listo para ingestar
hacer indice auto incremental en sql.

Se trabaja con el clima

In [57]:
df_list=[]
    
for file in glob.glob('**/new york 20*.csv'):
  df=pd.read_csv(file,delimiter = ',',encoding = "utf-8")
  df_list.append(df)
clima=pd.concat(df_list)

In [58]:
#clima = pd.read_csv("C:/Users/andre/OneDrive/Escritorio/proyecto final 3/Proyecto-Taxis-NYC/clima.csv")
clima.drop(columns=["name","dew","humidity","precipprob","preciptype","severerisk","uvindex","icon","stations"], inplace=True)
clima.drop(columns=["windgust","windspeed","winddir","sealevelpressure","cloudcover","visibility","solarradiation","solarenergy"], inplace=True)
clima["datetime"]=[x.replace('T'," ") for x in clima["datetime"]]
clima.datetime = clima['datetime'].apply(pd.to_datetime)

In [59]:
def t_clima():
    clima['temp'] = pd.to_numeric(clima['temp'], downcast="float" )
    clima['feelslike'] = pd.to_numeric(round(clima['feelslike'] ))
    clima['precip'] = pd.to_numeric(clima['precip'], downcast="float" )
    clima['snow'] = pd.to_numeric(clima['snow'], downcast="float" )
    clima['snowdepth'] = pd.to_numeric(clima['snowdepth'], downcast="float" )
    clima["conditions"]=clima["conditions"].astype(str)
    clima.rename(columns={"datetime": "hora"},inplace=True)
    #Creamos IdFecha
    clima["IdFecha"] = clima['hora'].dt.strftime('%Y%m%d%H')
    clima.IdFecha = clima.IdFecha.astype("int")
    #Cambio de variable
    clima['conditions']=np.where(clima['conditions']=="Clear",1,clima['conditions'])
    clima['conditions']=np.where(clima['conditions']=="Partially cloudy",2,clima['conditions'])
    clima['conditions']=np.where(clima['conditions']=="Overcast",3,clima['conditions'])
    clima['conditions']=np.where(clima['conditions']=="Snow, Overcast",4,clima['conditions'])
    clima['conditions']=np.where(clima['conditions']=="Snow, Partially cloudy",5,clima['conditions'])
    clima['conditions']=np.where(clima['conditions']=="Snow",6,clima['conditions'])
    clima['conditions']=np.where(clima['conditions']=="Rain, Overcast",7,clima['conditions'])
    clima['conditions']=np.where(clima['conditions']=="Rain, Partially cloudy",8,clima['conditions'])
    clima['conditions'] = pd.to_numeric(clima['conditions'], downcast="integer" )
    return clima

clima = t_clima()

Hasta aca trabajo con clima


Arrancamos con sql

In [60]:
import mysql.connector as msql
from mysql.connector import Error
from sqlalchemy import create_engine
from multiprocessing import connection

In [61]:
df = df_Impar

credenciales:


In [62]:
host_name,db_name, u_name, u_pass, port_num  = "127.0.0.1","taxi_2", "root", "Andres1", "3306"

In [63]:
#Creacion del motor de base de datos
engine = create_engine("mysql+mysqlconnector://" + u_name + ":" + u_pass + "@" 
                        + host_name + ":" + port_num + "/" + db_name, echo=False)

Creamos tablas para sql

Creamos tabla clima conditions

In [64]:
def clima_c():
    Clima_conditions = clima.IdCondiciones.unique()
    Clima_conditions = pd.DataFrame(Clima_conditions)
    Clima_conditions["conditions"] = Clima_conditions
    Clima_conditions = Clima_conditions.drop(columns=0)
    Clima_conditions.reset_index(inplace=True)
    Clima_conditions.rename(columns={"index":"IdCondiciones"},inplace=True)
    Clima_conditions.conditions.replace(1,"Clear", inplace=True)
    Clima_conditions.conditions.replace(2,"Partially cloudy", inplace=True)
    Clima_conditions.conditions.replace(3,"Overcast", inplace=True)
    Clima_conditions.conditions.replace(4,"Snow, Overcast", inplace=True)
    Clima_conditions.conditions.replace(5,"Snow, Partially cloudy", inplace=True)
    Clima_conditions.conditions.replace(6,"Snow", inplace=True)
    Clima_conditions.conditions.replace(7,"Rain, Overcast", inplace=True)
    Clima_conditions.conditions.replace(8,"Rain, Partially cloudy", inplace=True)
    Clima_conditions["IdCondiciones"] = Clima_conditions.apply(lambda row: row["IdCondiciones"]+1 , axis=1)

    Clima_conditions.to_sql(name="clima_conditions", con=engine, if_exists="append", index=False)

Creamos tabla Borough

In [65]:
def boro():
    Borough = pd.read_csv("Taxi+_zone_lookup.csv", usecols=['Borough'])
    Borough = Borough.Borough.unique()
    Borough = pd.DataFrame(Borough)
    Borough["Borough"] = Borough
    Borough = Borough.drop(columns=0)
    Borough.reset_index(inplace=True)
    Borough.rename(columns={"index":"IdBorough"},inplace=True)
    Borough["IdBorough"] = Borough.apply(lambda row: row["IdBorough"]+1 , axis=1)

    Borough.to_sql(name="borough", con=engine, if_exists="append", index=False)

Creamos tabla vendor

In [66]:
def vendor():
    Vendor = df_Impar.IdVendor.unique()
    Vendor = pd.DataFrame(Vendor)
    Vendor["IdVendor"] = Vendor
    Vendor = Vendor.drop(columns=0)
    Vendor["Vendor"] = Vendor.IdVendor
    Vendor.Vendor.replace(1,"Tecnologías móviles creativas", inplace=True)
    Vendor.Vendor.replace(2,"VeriFone Inc.", inplace=True)

    Vendor.to_sql(name="vendor", con=engine, if_exists="append", index=False)


Creamos tabla paymentType

In [67]:
def payment():
    PaymentType = df_Impar.IdPayment_Type.unique()
    PaymentType = pd.DataFrame(PaymentType)
    PaymentType["IdPayment_Type"] = PaymentType
    PaymentType = PaymentType.drop(columns=0)
    PaymentType["Payment"] = PaymentType.IdPayment_Type
    PaymentType.Payment.replace(1,"tarjeta de crédito", inplace=True)
    PaymentType.Payment.replace(2,"Efectivo", inplace=True)
    PaymentType.Payment.replace(3,"Sin cargo", inplace=True)
    PaymentType.Payment.replace(4,"Disputa", inplace=True)

    PaymentType.to_sql(name="payment_type", con=engine, chunksize=200, if_exists="append", index=False)

Creamos tabla rate code ID

In [68]:
def rate_code():
    RatecodeID = df_Impar.IdRatecode.unique()
    RatecodeID = pd.DataFrame(RatecodeID)
    RatecodeID["IdRatecode"] = RatecodeID
    RatecodeID = RatecodeID.drop(columns=0)
    RatecodeID["Ratecode"] = RatecodeID.IdRatecode
    RatecodeID.Ratecode.replace(1,"Tarifa estándar", inplace=True)
    RatecodeID.Ratecode.replace(2,"JFK", inplace=True)
    RatecodeID.Ratecode.replace(3,"Nueva York", inplace=True)
    RatecodeID.Ratecode.replace(4,"Nasáu o Westchester", inplace=True)
    RatecodeID.Ratecode.replace(5,"Tarifa negociada", inplace=True)

    RatecodeID.to_sql(name="ratecode", con=engine, if_exists="append", index=False)


In [69]:
clima= clima.rename(columns={'conditions':'IdCondiciones'}) #AGREGUE ESTA
clima.to_sql(name="clima_aux", con=engine, if_exists="append", index=False)



696

#ACOMODAR df_zone BOROUGH CON ID

In [70]:
def zone():
    df_zone=pd.read_csv("taxi+_zone_lookup.csv",sep=",")

    df_zone.rename(columns={"LocationID":"IdZona"},inplace=True)
    df_zone.rename(columns={"Borough":"IdBorough"},inplace=True)
    df_zone.IdBorough.replace("Queens",2, inplace=True)
    df_zone.IdBorough.replace("Bronx",3, inplace=True)
    df_zone.IdBorough.replace("Manhattan",4, inplace=True)
    df_zone.IdBorough.replace("Staten Island",5, inplace=True)
    df_zone.IdBorough.replace("Brooklyn",6, inplace=True)
    df_zone.IdBorough.replace("Unknown",7, inplace=True)
    df_zone.IdBorough.replace("EWR",1, inplace=True)

    df_zone.to_sql(name="zona", con=engine, if_exists="append", index=False)


Carga de tablas.

In [71]:
def switch_init():
    if os.path.isfile('delta.csv'):
        pass
    else:
        clima_c()
        boro()
        vendor()
        payment()
        rate_code()
        zone()     

switch_init()

In [72]:
df.to_csv("delta.csv",index=False, sep= ';')

In [73]:
def conexion(file):
    connection = msql.Connect(host=host_name, user=u_name, passwd=u_pass, db=db_name)
    cursor = connection.cursor()
    query = "LOAD DATA INFILE 'C:/Users/andre/OneDrive/Escritorio/proyecto final 3/Proyecto-Taxis-NYC/%s' INTO TABLE taxis_aux FIELDS TERMINATED BY ';' ENCLOSED BY '' ESCAPED BY '' IGNORE 1 LINES" % file
    cursor.execute(query)
    connection.commit()

conexion('delta.csv')



In [74]:
connection = msql.Connect(host=host_name, user=u_name, passwd=u_pass, db=db_name)
cursor = connection.cursor()
query = "CALL traerUnicos()" 
cursor.execute(query)
connection.commit()

In [75]:
connection = msql.Connect(host=host_name, user=u_name, passwd=u_pass, db=db_name)
cursor = connection.cursor()
query = "CALL traerUnicosClima()" 
cursor.execute(query)
connection.commit()

In [76]:
a=glob.glob('**/yellow_t*.parquet')
for i in range(0,len(a)):
  shutil.move(a[i], "data/cargados")

'data/cargados\\yellow_tripdata_2018-02.parquet'

In [77]:
a=glob.glob('**/new york 20*.csv')
for i in range(0,len(a)):
  shutil.move(a[i], "data/cargados")

'data/cargados\\new york 2018-01-31 to 2018-02-28.csv'